# `BMS.EchoProto.Protocol` Class

The `BMS.EchoProto.Protocol` class is used to capture all information relating to a single Echo protocol. The information captured by the `Protocol` object is used to generate the CSV Picklist files which are used to run the Echo.

## Creating an `EchoProto.Protocol`

First, the BMS generic tools are imported as BMS, and the EchoProto tools are imported as EchoProto

In [1]:
import BiomationScripter as BMS
from BiomationScripter import EchoProto

Next, a title for the Echo protocol is defined

In [2]:
Protocol_Title = "Example Protocol"

The `EchoProto.Protocol` object is created as shown below

In [3]:
Protocol = EchoProto.Protocol(
    Title = Protocol_Title
)
print(Protocol)

## Adding Source Plates

Source plates are plates which contain reagents or other liquids which will be transferred to the destination plate(s). Source plates are defined using the [`BMS.Labware_Layout`](../../BiomationScripter.md#class-labware_layout) class. These source plates can be newly created, or imported from [standard layout files](../../Standard_Layout_File.md) using the [`BMS.Import_Labware_Layout`](../../BiomationScripter.md#function-import_labware_layout) function.

Here, a DNA source plate is created.

In [4]:
Coloured_Water_Source_Layout = BMS.Labware_Layout(
    Name = "DNA_Source_Plate",
    Type = "384PP"
)

Coloured_Water_Source_Layout.add_content(
    Well = "A1",
    Reagent = "Red",
    Volume = 40,
    Liquid_Class = "AQ_BP"
)

Coloured_Water_Source_Layout.add_content(
    Well = "A2",
    Reagent = "Blue",
    Volume = 40,
    Liquid_Class = "AQ_BP"
)

When being used as a source plate for Echo protocols, the `BMS.Labware_Layout` MUST have a compatible source type defined. Compatible source types can be retrieved using `EchoProto.Source_Plate_Types.keys()`.

In [5]:
EchoProto.Source_Plate_Types.keys()

dict_keys(['384PP', '384LDV', '6RES'])

The source plate type used here is a 384PP plate

In [6]:
Coloured_Water_Source_Layout.type

'384PP'

Additionally, each reagent or other liquid in the source plate should have a liquid class specified. The liquid class is used by the Echo to determine the calibration to use for transfer events.

In [7]:
Coloured_Water_Source_Layout.print()

Information for DNA_Source_Plate
Plate Type: 384PP
Well	Volume(uL)	Liquid Class	Reagent
A1	40.0		AQ_BP		Red
A2	40.0		AQ_BP		Blue


'A1\t40.0\t\tAQ_BP\t\tRed\nA2\t40.0\t\tAQ_BP\t\tBlue\n'

A source plate can be added to an `EchoProto.Protocol` object using the `add_source_plate` method.

In [8]:
Protocol.add_source_plate(Coloured_Water_Source_Layout)

Source plates can be retrieved using the `get_source_plates` method

In [9]:
Protocol.get_source_plates()

## Adding Destination Plates

Destination plates define not only the type of plate to be used, but also the intended final contents after the protocol has been ran. Below, a destination plate is set up. Note that here, liquid classes are not required. This is because `EchoProto` uses the source plates to determine which liquid class to use for each liquid.

In [10]:
Destination_Plate_Layout = BMS.Labware_Layout(
    Name = "Destination Plate",
    Type = "Optical Amp Plate"
)
Destination_Plate_Layout.define_format(
    Rows = 16,
    Columns = 24
)

In [11]:
Destination_Plate_Layout.add_content(
    Well = "B2",
    Reagent = "Blue",
    Volume = 1
)

Destination_Plate_Layout.add_content(
    Well = "B2",
    Reagent = "Yellow",
    Volume = 1
)

Destination_Plate_Layout.add_content(
    Well = "B3",
    Reagent = "Blue",
    Volume = 1
)

Destination_Plate_Layout.add_content(
    Well = "B3",
    Reagent = "Red",
    Volume = 1
)

Destination_Plate_Layout.add_content(
    Well = "B4",
    Reagent = "Yellow",
    Volume = 1
)

Destination_Plate_Layout.add_content(
    Well = "B4",
    Reagent = "Red",
    Volume = 1
)

Destination_Plate_Layout.print()

Information for Destination Plate
Plate Type: Optical Amp Plate
Well	Volume(uL)	Liquid Class	Reagent
B2	1.0		Unknown		Blue
B2	1.0		Unknown		Yellow
B3	1.0		Unknown		Blue
B3	1.0		Unknown		Red
B4	1.0		Unknown		Yellow
B4	1.0		Unknown		Red


'B2\t1.0\t\tUnknown\t\tBlue\nB2\t1.0\t\tUnknown\t\tYellow\nB3\t1.0\t\tUnknown\t\tBlue\nB3\t1.0\t\tUnknown\t\tRed\nB4\t1.0\t\tUnknown\t\tYellow\nB4\t1.0\t\tUnknown\t\tRed\n'

The destination plate can be added using the `add_destination_plate` method, and retrieved using the `get_destination_plates` method, as shown below

In [12]:
Protocol.add_destination_plate(Destination_Plate_Layout)
Protocol.get_destination_plates()

## Making Echo Picklists from an `EchoProto.Protocol` Object

The [`EchoProto.Generate_Actions`]() and [`EchoProto.Write_Picklists`]() functions are used to create Picklists which can be imported to the Echo software and used to run the protocol defined here.

`EchoProto.Generate_Actions` is the first function to be used. This function will firstly check that all reagents/liquids specified in the destination plate(s) are present in a source plate. If there are any missing, an error is raised.

In [13]:
EchoProto.Generate_Actions(Protocol)

OutOFSourceMaterial: Cannot find the following reagents in a source plate: {'Yellow'}

As can be seen above, the "Yellow" liquid is missing. This can be added to the source plate as below

In [14]:
Coloured_Water_Source_Layout.add_content(
    Well = "A3",
    Reagent = "Yellow",
    Volume = 10,
    Liquid_Class = "AQ_BP"
)

After checking that all required reagents are present, `Generate_Actions` will then check that enough of all source material is present. This also takes into account the dead volume for the different source plate types.

In [15]:
EchoProto.Generate_Actions(Protocol)

Well A3 of plate DNA_Source_Plate containing Yellow is 5.0 uL below the dead volume.


OutOFSourceMaterial: 
Not enough volume of Yellow. 2.0 uL more is required. Last well checked was A3 of DNA_Source_Plate. This well is also 5.0 uL below the dead volume.


Name,Well,Plate,Volume Needed
Yellow,A3,DNA_Source_Plate,7.0


As can be seen above, the "Yellow" liquid does not have enough volume to complete the protocol. The error message indicates that 7 uL more is required. This volume is determined by calclating how much more liquid is required to ensure the reagent is above the dead volume (in this case, 5 uL), and the adding the amount of liquid required by the protocol (in this case, an extra 2 uL). This extra volume is added below

In [16]:
Current_Yellow_Volume = Coloured_Water_Source_Layout.get_volume_of_liquid_in_well("Yellow", "A3")

Coloured_Water_Source_Layout.update_volume_in_well(
    Well = "A3",
    Reagent = "Yellow",
    Volume = Current_Yellow_Volume + 7
)

Once all source material has been validated, the `Generate_Actions` function will determine the required liquid transfer events required to complete the protocol. The liquid transfer events are captured by `TransferList` objects, where each `TransferList` defines transfer events from one source plate. These are stored in the `EchoProto.Protocol` object using the `transfer_lists` attribute

In [17]:
EchoProto.Generate_Actions(Protocol)
print(Protocol.transfer_lists)

[[<BiomationScripter.EchoProto.TransferList object at 0x0000014FCAF82D00>]]


Following the creation of the Transfer Lists, the `Write_Picklists` function can be used to convert the `EchoProto.Protocol` object to a set of CSV picklist files. The `Write_Picklists` function takes three arguments. The first is the `Protocol` object to converted. The second is the location where the picklists should be saved. The third, `merge`, determined whether a picklist should be created for each source plate, or each source plate type. By default, `merge` is set to `False` and a separate picklist is created for every source plate, if there is more than one.

In [19]:
EchoProto.Write_Picklists(
    Protocol = Protocol,
    Save_Location = "Resources/",
    Merge = False
)

Resources//Example Protocol-384PP-(DNA_Source_Plate).csv


The generated picklist can be seen [here](Resources/Example%20Protocol-384PP-(DNA_Source_Plate).csv)